In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict

df = pd.read_csv('./developer_survey_2020/survey_results_public.csv')
schema_df = pd.read_csv('./developer_survey_2020/survey_results_schema.csv',encoding= 'unicode_escape')
df.head()

,Respondent,MainBranch,Hobbyist,Age,Age1stCode,CompFreq,CompTotal,ConvertedComp,Country,CurrencyDesc,...,SurveyEase,SurveyLength,Trans,UndergradMajor,WebframeDesireNextYear,WebframeWorkedWith,WelcomeChange,WorkWeekHrs,YearsCode,YearsCodePro
0,1,I am a developer by profession,Yes,NaN,13,Monthly,NaN,NaN,Germany,European Euro,...,Neither easy nor difficult,Appropriate in length,No,"Computer science, computer engineering, or sof...",ASP.NET Core,ASP.NET;ASP.NET Core,Just as welcome now as I felt last year,50.0,36,27
1,2,I am a developer by profession,No,NaN,19,NaN,NaN,NaN,United Kingdom,Pound sterling,...,NaN,NaN,NaN,"Computer science, computer engineering, or sof...",NaN,NaN,Somewhat more welcome now than last year,NaN,7,4
2,3,I code primarily as a hobby,Yes,NaN,15,NaN,NaN,NaN,Russian Federation,NaN,...,Neither easy nor difficult,Appropriate in length,NaN,NaN,NaN,NaN,Somewhat more welcome now than last year,NaN,4,NaN
3,4,I am a developer by profession,Yes,25.0,18,NaN,NaN,NaN,Albania,Albanian lek,...,NaN,NaN,No,"Computer science, computer engineering, or sof...",NaN,NaN,Somewhat less welcome now than last year,40.0,7,4
4,5,"I used to be a developer by profession, but no...",Yes,31.0,16,NaN,NaN,NaN,United States,NaN,...,Easy,Too short,No,"Computer science, computer engineering, or sof...",Django;Ruby on Rails,Ruby on Rails,Just as welcome now as I felt last year,NaN,15,8


In [36]:
df = df.dropna(subset= ['DevType'])

## Objective:  To Do -- create tests for the functions

In [73]:
data_jobs = ['Data or business analyst', 'Data or business analyst', 'Engineer, data']

In [74]:
def job_filter(devtype, data_jobs):
    #data_jobs = ['Data or business analyst', 'Data or business analyst', 'Engineer, data']
    for job in data_jobs:
        if job in devtype:
            return 1
    return 0

In [75]:
test1 =  'Developer, back-end;Educator'
test2 = 'Database administrator;Developer, back-end;Developer, front-end;Developer, full-stack'
test3 = 'Data scientist or machine learning specialist;Developer, back-end;Engineer, data'
job_filter(test1, data_jobs)
job_filter(test2, data_jobs)
job_filter(test3, data_jobs)

1

## How many people answering the survey have data science related jobs?

In [76]:
df['DevType'].apply(job_filter, data_jobs= data_jobs).sum()

6653

In [77]:
df['dataJob'] = df['DevType'].apply(job_filter, data_jobs= data_jobs)

/Users/hananather/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


## Select data related jobs:

In [78]:
data_jobs_df = df[df['dataJob'] == 1] 

In [79]:
def clean_counts(df, col, values, col1, col2 = 'counts'):
    
    raw_df = df[col].value_counts().reset_index()
    clean_df = defaultdict(int)
    
    for val in values:    
        for index, row in raw_df.iterrows():
            if val in list(row)[0]:
                clean_df[val] += int(list(row)[1])
    
    clean_df = pd.DataFrame(pd.Series(clean_df)).reset_index()
    clean_df.columns = [col1, col2]
    clean_df = clean_df.sort_values(col2, ascending= False).reset_index(drop=True)
    
    return clean_df

# What are the most popular programing languages being used by data scientist, data analysts and data engineers in 2020?

In [80]:
languages = ['Assembly', 'Bash/Shell/PowerShell', 'C','C#','C++','Dart' ,'Go', 'Haskell',
             'HTML/CSS', 'Java', 'JavaScript', 'Julia','Kotlin', 'Objective-C', 'Perl',
             'PHP', 'Python', 'R']

In [81]:
clean_counts(data_jobs_df,'LanguageWorkedWith', languages, 'language')

,language,counts
0,C,4615
1,Java,4482
2,Python,3827
3,JavaScript,3798
4,HTML/CSS,3686
5,Bash/Shell/PowerShell,2538
6,C#,1936
7,PHP,1601
8,R,1546
9,C++,1387


In [ ]:
Wha

In [82]:
clean_counts(data_jobs_df,'LanguageDesireNextYear', languages, 'language')

,language,counts
0,Python,3714
1,C,3345
2,Java,3159
3,JavaScript,2614
4,R,2285
5,HTML/CSS,2068
6,Bash/Shell/PowerShell,1601
7,C#,1461
8,Go,1423
9,C++,999


# What are the most popular database environments used by Data Scientists in 2020?

In [83]:
databases = ['Cassandra', 'Couchbase', 'DynamoDB', 'Elasticsearch', 'Firebase',
             'IBM DB2', 'MariaDB', 'Microsoft', 'SQL Server', 'MongoDB', 'MySQL', 
             'Oracle']

In [85]:
clean_counts(data_jobs_df,'DatabaseWorkedWith', databases, 'database')

,database,counts
0,MySQL,3253
1,Microsoft,2610
2,SQL Server,2610
3,MongoDB,1507
4,Oracle,1306
5,MariaDB,1067
6,Elasticsearch,1018
7,Firebase,680
8,DynamoDB,494
9,Cassandra,334


# What are the most desired databases for next year?

In [86]:
clean_counts(data_jobs_df,'DatabaseDesireNextYear', databases, 'database')

,database,counts
0,MySQL,1890
1,MongoDB,1746
2,Microsoft,1602
3,SQL Server,1602
4,Elasticsearch,1452
5,Oracle,826
6,Firebase,822
7,MariaDB,801
8,Cassandra,777
9,DynamoDB,701


# What are the most used platforms by data scientists in 2020?

In [87]:
platforms = ['Android','Arduino', 'AWS', 'Docker', 'Google Cloud Platform', 'Heroku', 
             'IBM Cloud or Watson iOS', 'Kubernetes', 'Linux', 'MacOS', 'Microsoft Azure', 
             'Raspberry Pi', 'Slack Apps and Integrations Windows'] 

In [88]:
clean_counts(data_jobs_df,'PlatformWorkedWith', platforms, 'platforms')

,platforms,counts
0,Linux,3562
1,Docker,2309
2,AWS,2046
3,MacOS,1445
4,Android,1331
5,Google Cloud Platform,1217
6,Microsoft Azure,1211
7,Raspberry Pi,1046
8,Kubernetes,847
9,Arduino,708


# What platforms do data scientists want to work with NEXT year?

In [89]:
clean_counts(data_jobs_df,'PlatformDesireNextYear', platforms, 'platforms')

,platforms,counts
0,Linux,3218
1,Docker,2768
2,AWS,2432
3,Kubernetes,1791
4,Google Cloud Platform,1709
5,Microsoft Azure,1495
6,Raspberry Pi,1486
7,Android,1462
8,MacOS,1270
9,Arduino,823


# what are the most used frameworks for data scientists?

In [90]:
other_framworks = ['.NET', '.NET', 'Core', 'Ansible', 'Apache Spark', 'Chef', 
                 'Cordova', 'Flutter', 'Hadoop', 'Keras', 'Node.js', 'Pandas', 
                 'Puppet', 'React', 'Native', 'TensorFlow', 'Terraform', 'Torch/PyTorch', 'Unity', 
                 '3D Unreal Engine'] 

In [91]:
clean_counts(data_jobs_df,'MiscTechWorkedWith', other_framworks, 'other_framworks')

,other_framworks,counts
0,.NET,3676
1,Node.js,1974
2,Pandas,1869
3,Core,1189
4,TensorFlow,971
5,Apache Spark,813
6,Hadoop,685
7,Keras,577
8,React,447
9,Native,447


# what are the most desired frameworks for data scientists for the future?

In [92]:
clean_counts(data_jobs_df,'MiscTechDesireNextYear', other_framworks, 'other_framworks')

,other_framworks,counts
0,.NET,3086
1,TensorFlow,2089
2,Node.js,1920
3,Pandas,1874
4,Core,1270
5,Apache Spark,1226
6,Torch/PyTorch,1116
7,Hadoop,1050
8,Keras,899
9,React,886
